In [1]:
from pylab import rcParams
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import os 
import sys
import numpy as np
import pandas as pd

import pyarrow as pa
import pyarrow.parquet as pq
import h5py

import os,glob 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/My\ Drive/Falcon_pixel_layers/

/content/drive/My Drive/Falcon_pixel_layers


In [4]:
# Open the input HDF5 file
dsets = h5py.File('./HDF5_dataset/BoostedJets_fullSample_x1_file-0.hdf5', 'r')
keys = ['X_jets','y'] # key names in in put hdf5


In [5]:

def np2arrowArray(x):
    if len(x.shape) > 1:
        x = np.transpose(x, [2,0,1])
        return pa.array([x.tolist()])
    else:
        return pa.array([x.tolist()])


def convert_to_Parquet(filename, start, stop, chunk_size):
    # Open the input HDF5 file
    dsets = h5py.File(filename, 'r')
    keys = ['X_jets','y'] # key names in input hdf5
    row0 = [np2arrowArray(dsets[key][0]) for key in keys]
    keys = ['X_jets','y'] # desired key names in output parquet
    table0 = pa.Table.from_arrays(row0, keys) 
    
    # Open the output Parquet file
    filename = './Parquet_Data/%s.snappy.parquet'%(filename[16:-5])
    writer = pq.ParquetWriter(filename, table0.schema, compression='snappy')

    # Loop over file chunks of size chunk_size
    nevts = stop - start
    for i in range(int(np.ceil(1.*nevts/chunk_size))):
        
        begin = start + i*chunk_size
        end = begin + chunk_size

        # Load array chunks into memory
        X = [dsets[key][begin:end] for key in keys]
        # Convert events in the chunk one-by-one
        print (filename)
        print('Doing events: [%d->%d)'%(begin,end))
        print(len(X[0][0].shape))
        for j in range(len(X[0])):

            # Create a list for each sample
            sample=[np2arrowArray(X[key][j]) for key in range(len(keys))]       
            checkY=X[1][j]
            #print(checkY)
            if checkY==1:
              table = pa.Table.from_arrays(sample, keys)

              writer.write_table(table)

    writer.close()
    return filename


In [6]:
with h5py.File('./HDF5_dataset/BoostedJets_fullSample_x1_file-0.hdf5', "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[3]

    # Get the data
    data = list(f[a_group_key])

Keys: <KeysViewHDF5 ['X_jets', 'm0', 'pt', 'y']>


In [7]:
dataray=np.array(data)

tops=np.where(dataray==1)[0]


In [8]:
tops=tops.tolist()

In [10]:
wdir = r'Parquet_Data'
os.chdir(wdir)

In [6]:

#hdf5_filenames = ['ADD THE FILE NAMES-%s.h5'%i for i in range(0, ADD THE RANGE)]
hdf5_filenames = glob.glob("./HDF5_dataset/*")
chunk_size = 3200
start = 0 
stop = chunk_size*10
for filename in hdf5_filenames:
  convert_to_Parquet(filename, start, stop, chunk_size)


#if __name__ == "__main__":
 #   main()


./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [0->3200)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [3200->6400)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [6400->9600)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [9600->12800)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [12800->16000)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [16000->19200)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [19200->22400)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [22400->25600)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [25600->28800)
3
./Parquet_Data/oostedJets_fullSample_x1_file-16.snappy.parquet
Doing events: [28800->32000)
3
./Parquet_Data/oostedJets_fullSample_x1_file-18.snappy.parquet
Doing e

In [ ]:
pwd

'/content/drive/My Drive/Falcon_pixel_layers'